<a href="https://colab.research.google.com/github/MayAnhhh163/NCKH/blob/main/SVM%2BRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style
import seaborn as sns
import nltk #Natural Language Toolkit
#To ignore warnings
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
!pip install nltk
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [ ]:
# Đường dẫn đến file CSV trên Google Drive (điều chỉnh theo vị trí file của bạn)
file_path = '/content/drive/Shared drives/NCKH/DATA/Data_scrap_merged1.csv'

df = pd.read_csv(file_path)
print(df.head())

       File Name                                              Title  \
0  10000847.html  Wells Fargo – Banking, Credit Cards, Loans, Mo...   
1  10000855.html  La Banque Postale - Banque et Assurance en lig...   
2  10000856.html                                   LCL - Mon espace   
3  10000946.html                                           No title   
4  10000976.html                                           Facebook   

                                             Content  Result  
0  Wells Fargo – Banking, Credit Cards, Loans, Mo...     1.0  
1  La Banque Postale - Banque et Assurance en lig...     1.0  
2  LCL - Mon espace\nVotre identifiant\nMémoriser...     1.0  
3  ov&-5qtm_^yDKx!FcUWSM^AK2%9bB9~4O>4#!MXy5N3-Mp...     1.0  
4  Facebook\nMessenger\nAlbin Berisha\nAlbin shar...     1.0  


In [ ]:
df.head()


,File Name,Title,Content,Result
0,10000847.html,"Wells Fargo – Banking, Credit Cards, Loans, Mo...","Wells Fargo – Banking, Credit Cards, Loans, Mo...",1.0
1,10000855.html,La Banque Postale - Banque et Assurance en lig...,La Banque Postale - Banque et Assurance en lig...,1.0
2,10000856.html,LCL - Mon espace,LCL - Mon espace\nVotre identifiant\nMémoriser...,1.0
3,10000946.html,No title,ov&-5qtm_^yDKx!FcUWSM^AK2%9bB9~4O>4#!MXy5N3-Mp...,1.0
4,10000976.html,Facebook,Facebook\nMessenger\nAlbin Berisha\nAlbin shar...,1.0


In [ ]:
df.shape

(75038, 4)

In [ ]:
df.columns

Index(['File Name', 'Title', 'Content', 'Result'], dtype='object')

In [ ]:
df = df.dropna()

Tag removes

In [ ]:
def remove_tags(text):
    # Sử dụng BeautifulSoup để loại bỏ thẻ HTML
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

Stop Words

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

Punctuation Removal

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

Lower

In [ ]:
def to_lowercase(text):
    return text.lower()

Lemmatization:

In [ ]:
lemmatizer = WordNetLemmatizer()

def apply_lemmatization(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)


Tokenization

In [ ]:
def tokenize_text(text):
    return word_tokenize(text)

Tổng bước 2:

In [ ]:
def preprocess_text(text):
    # Bước 1: Loại bỏ thẻ HTML
    text = remove_tags(text)
    # Bước 2: Loại bỏ stop words
    text = remove_stop_words(text)
    # Bước 3: Loại bỏ dấu câu
    text = remove_punctuation(text)
    # Bước 4: Chuyển về chữ thường
    text = to_lowercase(text)
    # Bước 5: Áp dụng Lemmatization
    text = apply_lemmatization(text)
    # Bước 6: Tokenization
    tokens = tokenize_text(text)
    return tokens

đọc

In [ ]:
data = df.copy()

In [ ]:
data = data.dropna(subset=['Content'])

In [ ]:
from multiprocessing import cpu_count
print(cpu_count())  # Số lõi CPU khả dụng (bao gồm các luồng xử lý)

2


In [ ]:
import os
print(os.cpu_count())  # Số lõi CPU khả dụng (bao gồm các luồng xử lý)

2


In [ ]:
chunksize = 10000  # Số dòng xử lý mỗi lần
processed_chunks = []

# Giới hạn số dòng xử lý (nếu cần, ví dụ chỉ lấy tối đa 500,000 dòng đầu tiên)
max_rows = 50000  # Số dòng tối đa cần xử lý

for i in range(0, min(len(data), max_rows), chunksize):  # Lặp qua các chunk
    chunk = data.iloc[i:i + chunksize].copy()  # Trích xuất chunk, dùng .copy() để tránh cảnh báo SettingWithCopyWarning
    chunk['processed_content'] = chunk['Content'].apply(preprocess_text)  # Áp dụng hàm xử lý
    processed_chunks.append(chunk)

# Gộp lại tất cả các phần đã xử lý
df_processed = pd.concat(processed_chunks, ignore_index=True)  # Sử dụng ignore_index để làm mới chỉ mục

# Kiểm tra kết quả
print(df_processed.head())
print(f"Processed rows: {len(df_processed)}")

       File Name                                              Title  \
0  10000847.html  Wells Fargo – Banking, Credit Cards, Loans, Mo...   
1  10000855.html  La Banque Postale - Banque et Assurance en lig...   
2  10000856.html                                   LCL - Mon espace   
3  10000946.html                                           No title   
4  10000976.html                                           Facebook   

                                             Content  Result  \
0  Wells Fargo – Banking, Credit Cards, Loans, Mo...     1.0   
1  La Banque Postale - Banque et Assurance en lig...     1.0   
2  LCL - Mon espace\nVotre identifiant\nMémoriser...     1.0   
3  ov&-5qtm_^yDKx!FcUWSM^AK2%9bB9~4O>4#!MXy5N3-Mp...     1.0   
4  Facebook\nMessenger\nAlbin Berisha\nAlbin shar...     1.0   

                                   processed_content  
0  [well, fargo, banking, credit, card, loan, mor...  
1  [la, banque, postale, banque, et, assurance, e...  
2  [lcl, mon, espace, v

In [ ]:
df_processed = df_processed.dropna()

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import vstack
import pandas as pd

df_processed['processed_content'] = df_processed['processed_content'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else x
)

# Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # 1-gram và 2-gram

# Kích thước mỗi chunk
chunk_size = 100000  # Chia thành các phần 500,000 dòng

# Áp dụng TF-IDF theo từng chunk
chunks = [df_processed[i:i + chunk_size] for i in range(0, df_processed.shape[0], chunk_size)]
tfidf_matrices = []

for chunk in chunks:
    X_chunk = tfidf_vectorizer.fit_transform(chunk['processed_content'])
    tfidf_matrices.append(X_chunk)

# Kết hợp tất cả các ma trận TF-IDF lại
X_tfidf = vstack(tfidf_matrices)
print(f"Kích thước ma trận TF-IDF: {X_tfidf.shape}")

Kích thước ma trận TF-IDF: (50000, 5000)


# thực hiện model


# RandomForest

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Number of legitimate website instances (labelled as 0 in the SQL file): 50,000
#Number of phishing website instances (labelled as 1 in the SQL file): 30,000
# Giả sử y là cột nhãn (0: URL xấu, 1: URL tốt)
X = X_tfidf  # Ma trận TF-IDF
y = df_processed['Result']  # Nhãn đã được chuẩn bị

# Chia dữ liệu thành 80% train và 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Khởi tạo mô hình Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators: số lượng cây

# Huấn luyện mô hình
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_test)

# Tính độ chính xác
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Báo cáo chi tiết
print(classification_report(y_test, y_pred))

Accuracy: 96.00%
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97      6436
         1.0       0.95      0.94      0.94      3564

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



#SVM


In [ ]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
X_tfidf, df_processed['Result'], test_size=0.2, random_state=42
)

print(f"Số lượng mẫu trong tập huấn luyện: {X_train.shape}")
print(f"Số lượng mẫu trong tập kiểm tra: {X_test.shape}")

Số lượng mẫu trong tập huấn luyện: (40000, 5000)
Số lượng mẫu trong tập kiểm tra: (10000, 5000)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

# Khởi tạo mô hình SVM
svm_model = LinearSVC(random_state=42)

# Huấn luyện mô hình trên tập huấn luyện
svm_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = svm_model.predict(X_test)

# Đánh giá hiệu suất mô hình
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))
print(f"Độ chính xác: {accuracy_score(y_test, y_pred):.4f}")

Báo cáo phân loại:
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.96      6436
         1.0       0.92      0.94      0.93      3564

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Độ chính xác: 0.9516


In [ ]:
from sklearn.model_selection import GridSearchCV

# Định nghĩa các tham số cần tối ưu
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'max_iter': [1000, 2000, 3000]
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(
    LinearSVC(random_state=42), param_grid, cv=5, scoring='accuracy'
)

# Tìm kiếm tham số tối ưu
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_:.4f}")

Best parameters: {'C': 1, 'max_iter': 1000}
Best cross-validation accuracy: 0.9509


In [ ]:
import joblib

# Lưu mô hình
joblib.dump(svm_model, 'svm_model.pkl')

# Tải lại mô hình khi cần thiết
loaded_model = joblib.load('svm_model.pkl')

# Sử dụng mô hình để dự đoán
new_predictions = loaded_model.predict(X_test)

#word2vec và TF-IDF


In [ ]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Khởi tạo Word2Vec và TF-IDF
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)  # Skip-Gram model
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences_as_strings)

# 2. Lấy từ điển TF-IDF
tfidf_vocab = tfidf_vectorizer.vocabulary_
tfidf_weights = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))

# 3. Kết hợp Word2Vec và TF-IDF
def sentence_vector(sentence, word2vec_model, tfidf_weights):
    vector_size = word2vec_model.vector_size
    words = sentence.split()
    tfidf_weighted_vectors = []

    for word in words:
        if word in word2vec_model.wv and word in tfidf_weights:
            tfidf_weight = tfidf_weights[word]
            word_vector = word2vec_model.wv[word]
            tfidf_weighted_vectors.append(tfidf_weight * word_vector)

    if tfidf_weighted_vectors:
        return np.mean(tfidf_weighted_vectors, axis=0)  # Vector trung bình
    else:
        return np.zeros(vector_size)

# Tính vector cho từng câu
sentence_vectors = [sentence_vector(sentence, word2vec_model, tfidf_weights) for sentence in sentences_as_strings]


NameError: name 'sentences' is not defined

In [ ]:
kkkkkkkk
